## Analysis with Display

In [96]:
import pandas as pd
import numpy as np
import requests
from keras.models import load_model
from keras.preprocessing import image
import pymysql as sql

In [21]:
df=pd.read_csv('hist_data.csv', encoding="cp1252")
model = load_model('nsfw.h5')
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [184]:
def rateurl(i):
            BASE = 'https://render-tron.appspot.com/screenshot/'
            path = 'target.jpg'
            response = requests.get(BASE + i, stream=True)
            if response.status_code != 200:
                return "unknown"
            if response.status_code == 200:
                with open(path, 'wb') as file:
                    for chunk in response:
                        file.write(chunk)
            test_image = image.load_img('target.jpg', target_size = (64, 64)) 
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image, axis = 0)
            result = model.predict(test_image)
            categories=['safe','not safe']
            status=categories[int(result[0][0])]
            url_dict[i]=categories[int(result[0][0])]
            return status


In [175]:
def statuscheck(url):
    c=sql.connect(host="localhost",user="root",password="",db="urls")
    connection=c.cursor()
    command="SELECT status FROM urls.url_status where name='"+ url +"';"
    connection.execute(command)
    retrived=connection.fetchall()
    return retrived

def urlinsert(url):
    c=sql.connect(host="localhost",user="root",password="",db="urls")
    connection=c.cursor()
    status=rateurl(url)
    command="insert into urls.url_status values('" +url+ "','" +status+ "');"
    connection.execute(command)
    return status

In [176]:
def sync(url):
    status=statuscheck(url)
    if len(status)!=0:
        return status
    else:
        return urlinsert(url)

In [252]:
def result(url):
    status=str(url)
    status=status.split()
    status=status[1]
    status=sync(status)
    if type(status)==str:
        return status
    if type(status)==tuple:
        return status[0][0]

In [ ]:
urlscan=df.iloc[:,[0]].apply(result,axis=1)
urlscan


In [256]:
dataframe=pd.DataFrame(urlscan)
dataframe.to_csv('scannedurl.csv')